In [29]:
import pandas as pd
import pandas as pd
file_path = "netflix_titles.csv"

data = pd.read_csv(file_path,sep=",",header=0,encoding="utf-8",index_col=False)
data = data.dropna()

def create_new_csv_no(name, data):
    data.to_csv(name, sep=",", encoding="utf-8",index=False)
def create_new_csv(name, data):
    data.to_csv(name, sep=",", encoding="utf-8")


movie = data[['show_id','title', 'description','duration','type']]
movie= movie[movie['type'] == 'Movie']
movie = movie.drop(columns=['type'])
movie['duration'] = movie['duration'].str.extract('(\d+)', expand=False).astype(int)
create_new_csv_no("movie.csv", movie)

series = data[['show_id','title', 'description','duration','type']]
series= series[series['type'] == 'TV Show']
series = series.drop(columns=['type'])
series['duration'] = series['duration'].str.extract('(\d+)', expand=False).astype(int)
create_new_csv_no("series.csv", series)


person1 = pd.DataFrame(data['cast'].str.split(',').explode().str.strip())
person1.columns = ['person']
person2 = pd.DataFrame(data['director'].str.split(',').explode().str.strip())
person2.columns = ['person']

person = pd.concat([person1, person2])
person.drop_duplicates(subset="person", keep='first', inplace=True)

create_new_csv_no("person.csv", person)

date = data[['date_added']]
date['month'] = date['date_added'].str.split(' ').str[0]
date['day'] = date['date_added'].str.split(' ').str[1].str[:-1]
date['year'] = date['date_added'].str.split(', ').str[1]
date = date.drop(columns=['date_added'])

create_new_csv_no("date.csv", date)


print(date)




          month day  year
7     September  24  2021
8     September  24  2021
9     September  24  2021
12    September  23  2021
24    September  21  2021
...         ...  ..   ...
8801      March   9  2016
8802   November  20  2019
8804   November   1  2019
8805    January  11  2020
8806      March   2  2019

[5332 rows x 3 columns]


C:\Users\leiya\AppData\Local\Temp\ipykernel_16620\1910296626.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date['month'] = date['date_added'].str.split(' ').str[0]
C:\Users\leiya\AppData\Local\Temp\ipykernel_16620\1910296626.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date['day'] = date['date_added'].str.split(' ').str[1].str[:-1]
C:\Users\leiya\AppData\Local\Temp\ipykernel_16620\1910296626.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try